In [1]:
import os
import pandas as pd
from pathlib import Path

# Define the directory path
data_dir = Path("../aligned_timeseries_FINS")

# Get all CSV files matching the pattern
matching_files = [f for f in data_dir.glob("FINS_*c_*_filtered.csv")]

# List to store files with post_chat
files_with_post_chat = []

# Check each file for post_chat in condition column
for file_path in matching_files:
    try:
        df = pd.read_csv(file_path)
        if 'condition' in df.columns:
            if 'post_chat' in df['condition'].values:
                files_with_post_chat.append(file_path.name)
    except Exception as e:
        print(f"Error reading {file_path.name}: {e}")

# Print results
print(f"Total matching files: {len(matching_files)}")
print(f"Files with 'post_chat': {len(files_with_post_chat)}")
print("\nFiles containing 'post_chat' in condition column:")
for filename in sorted(files_with_post_chat):
    print(f"  - {filename}")

Total matching files: 44
Files with 'post_chat': 41

Files containing 'post_chat' in condition column:
  - FINS_004c_aligned_timeseries_filtered.csv
  - FINS_006c_aligned_timeseries_filtered.csv
  - FINS_007c_aligned_timeseries_filtered.csv
  - FINS_008c_aligned_timeseries_filtered.csv
  - FINS_009c_aligned_timeseries_filtered.csv
  - FINS_010c_aligned_timeseries_filtered.csv
  - FINS_011c_aligned_timeseries_filtered.csv
  - FINS_013c_aligned_timeseries_filtered.csv
  - FINS_014c_aligned_timeseries_filtered.csv
  - FINS_015c_aligned_timeseries_filtered.csv
  - FINS_016c_aligned_timeseries_filtered.csv
  - FINS_017c_aligned_timeseries_filtered.csv
  - FINS_018c_aligned_timeseries_filtered.csv
  - FINS_019c_aligned_timeseries_filtered.csv
  - FINS_021c_aligned_timeseries_filtered.csv
  - FINS_022c_aligned_timeseries_filtered.csv
  - FINS_023c_aligned_timeseries_filtered.csv
  - FINS_024c_aligned_timeseries_filtered.csv
  - FINS_025c_aligned_timeseries_filtered.csv
  - FINS_027c_aligned_t

In [2]:
sample_df = pd.read_csv('../aligned_timeseries_FINS/FINS_056c_aligned_timeseries_filtered.csv')

## Plan for fitting two independent HMMs (child1 HMM, child2 HMM), then compare state sequences

### Step 1 — Load both CSVs

### Step 2 — Filter to `post_chat`

### Step 3 — Select HbO columns only → build X

### Step 4 — EDA on filtered data

* duration (sec/min)
* sampling rate (~10 Hz)
* missingness rate (fraction of NaNs)
* number of HbO channels

### Step 5 — Standardize X (mean 0, std 1 per HbO channel)

### Step 6 — Fit HMM on each file (same K)

* start with K = 4..10
* pick K later by BIC/stability; for now pick a reasonable K like 6

### Step 7 — Align state labels across the two HMMs (Hungarian)

* align by similarity of state mean patterns

### Step 8 — Compare aligned outputs

* fractional occupancy per aligned state
* dwell time per aligned state

Focused on:

* `condition == "post_chat"`
* **HbO only**
* EDA: minutes, sampling rate, missingness
* standardization (recommended for HMM stability)
* fit HMM to each file
* align states between the two HMMs using **Hungarian algorithm**

In [12]:
!pip -q install hmmlearn scikit-learn scipy pandas numpy

2568.93s - pydevd: Sending message related to process being replaced timed-out after 5 seconds

[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [15]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM
from scipy.optimize import linear_sum_assignment

### EDA Helpers

In [16]:
def get_hbo_columns(df):
    # Matches columns ending with ' hbo' (case-insensitive)
    return [c for c in df.columns if c.lower().endswith(" hbo")]

def estimate_sampling_rate(time_sec_rel):
    t = np.asarray(time_sec_rel)
    dt = np.diff(t)
    dt = dt[np.isfinite(dt)]
    if len(dt) == 0:
        return np.nan, np.nan
    med_dt = float(np.median(dt))
    hz = (1.0 / med_dt) if med_dt > 0 else np.nan
    return med_dt, hz

def missingness_rate(X):
    # fraction of NaNs in the whole matrix
    return float(np.isnan(X).mean())

def eda_postchat_hbo(df, name="file"):
    if "condition" not in df.columns:
        raise ValueError(f"{name}: no 'condition' column found.")
    if "time_sec_rel" not in df.columns:
        raise ValueError(f"{name}: no 'time_sec_rel' column found.")
    
    d = df[df["condition"] == "post_chat"].copy()
    hbo_cols = get_hbo_columns(d)
    if len(hbo_cols) == 0:
        raise ValueError(f"{name}: no HbO columns found after filtering to post_chat.")
    
    X = d[hbo_cols].astype(float).values
    t = d["time_sec_rel"].values
    
    duration_sec = float(t[-1] - t[0]) if len(t) > 1 else 0.0
    duration_min = duration_sec / 60.0
    med_dt, hz = estimate_sampling_rate(t)
    miss = missingness_rate(X)
    
    print(f"\n=== EDA (post_chat, HbO only): {name} ===")
    print("Rows (time points):", len(d))
    print("HbO channels (D):", len(hbo_cols))
    print(f"Duration: {duration_sec:.2f} sec (~{duration_min:.2f} min)")
    print(f"Estimated sampling: dt≈{med_dt:.4f}s  ->  {hz:.2f} Hz")
    print(f"Missingness rate (NaN fraction): {miss:.6f}")
    
    return d, hbo_cols, X, t

### Loading files + run post_chat HbO EDA

In [17]:
file_a = f"{data_dir}/FINS_056c_aligned_timeseries_filtered.csv"
file_b = f"{data_dir}/FINS_057c_aligned_timeseries_filtered.csv"

df_a = pd.read_csv(file_a)
df_b = pd.read_csv(file_b)

d_a, hbo_a, Xa, ta = eda_postchat_hbo(df_a, name="FINS_056c")
d_b, hbo_b, Xb, tb = eda_postchat_hbo(df_b, name="FINS_057c")

# Ensure we use the same HbO channels in both (intersection if needed)
common_hbo = sorted(list(set(hbo_a).intersection(set(hbo_b))))
print("\nCommon HbO channels:", len(common_hbo))

Xa = d_a[common_hbo].astype(float).values
Xb = d_b[common_hbo].astype(float).values


=== EDA (post_chat, HbO only): FINS_056c ===
Rows (time points): 799
HbO channels (D): 22
Duration: 18.48 sec (~0.31 min)
Estimated sampling: dt≈0.0983s  ->  10.17 Hz
Missingness rate (NaN fraction): 0.000000

=== EDA (post_chat, HbO only): FINS_057c ===
Rows (time points): 688
HbO channels (D): 22
Duration: 7.57 sec (~0.13 min)
Estimated sampling: dt≈0.0983s  ->  10.17 Hz
Missingness rate (NaN fraction): 0.000000

Common HbO channels: 22


### Standardize

In [18]:
def standardize(X):
    scaler = StandardScaler()
    Xz = scaler.fit_transform(X)
    return Xz, scaler

Xa_z, scaler_a = standardize(Xa)
Xb_z, scaler_b = standardize(Xb)


### Fit HMM (no PCA for now, per your preference)

In [19]:
def fit_hmm(Xz, K, seed=0, n_iter=500):
    model = GaussianHMM(
        n_components=K,
        covariance_type="diag",
        n_iter=n_iter,
        tol=1e-3,
        random_state=seed
    )
    model.fit(Xz)
    z = model.predict(Xz)
    return model, z

K = 6  # starter value; we can sweep K later

model_a, z_a = fit_hmm(Xa_z, K=K, seed=0)
model_b, z_b = fit_hmm(Xb_z, K=K, seed=0)

### Hungarian alignment (align B’s states to A’s labels)
We align using cosine similarity of the state mean vectors (`model.means_`), which are in the standardized feature space here.

In [20]:
def cosine_similarity_matrix(A, B, eps=1e-9):
    A_norm = A / (np.linalg.norm(A, axis=1, keepdims=True) + eps)
    B_norm = B / (np.linalg.norm(B, axis=1, keepdims=True) + eps)
    return A_norm @ B_norm.T

def hungarian_align(means_a, means_b):
    S = cosine_similarity_matrix(means_a, means_b)
    cost = -S  # maximize similarity
    row_ind, col_ind = linear_sum_assignment(cost)
    # col_ind[a_state] = matched b_state
    return col_ind, S

def remap_states_b_to_a(z_b, mapping_a_to_b):
    # Build inverse map: inv[b_state] = a_state
    inv = np.zeros_like(mapping_a_to_b)
    for a_state, b_state in enumerate(mapping_a_to_b):
        inv[b_state] = a_state
    return inv[z_b]

mapping_a_to_b, sim = hungarian_align(model_a.means_, model_b.means_)
z_b_aligned = remap_states_b_to_a(z_b, mapping_a_to_b)

print("Mapping (A_state -> B_state):", mapping_a_to_b.tolist())
print("Avg matched similarity:", float(np.mean([sim[a, mapping_a_to_b[a]] for a in range(K)])))

Mapping (A_state -> B_state): [4, 5, 3, 2, 1, 0]
Avg matched similarity: 0.27999338086159187


### Compare aligned occupancy + dwell

In [21]:
def fractional_occupancy(z, K):
    return np.bincount(z, minlength=K) / len(z)

def mean_dwell(z, K):
    dw = {k: [] for k in range(K)}
    s, run = z[0], 1
    for i in range(1, len(z)):
        if z[i] == s:
            run += 1
        else:
            dw[s].append(run)
            s, run = z[i], 1
    dw[s].append(run)
    return np.array([np.mean(dw[k]) if dw[k] else 0.0 for k in range(K)])

fo_a = fractional_occupancy(z_a, K)
fo_b = fractional_occupancy(z_b_aligned, K)

dt_a = mean_dwell(z_a, K)
dt_b = mean_dwell(z_b_aligned, K)

print("\nFO A:", np.round(fo_a, 3))
print("FO B (aligned):", np.round(fo_b, 3))
print("Mean dwell A (steps):", np.round(dt_a, 1))
print("Mean dwell B (aligned, steps):", np.round(dt_b, 1))


FO A: [0.218 0.242 0.081 0.091 0.215 0.153]
FO B (aligned): [0.113 0.215 0.126 0.188 0.166 0.192]
Mean dwell A (steps): [ 87.   96.5  32.5  24.3  57.3 122. ]
Mean dwell B (aligned, steps): [ 26.  148.   87.   21.5  28.5  22. ]


### DEBUG PER FILE post_chat Length

In [ ]:
# file_a = f"{data_dir}/FINS_040c_aligned_timeseries_filtered.csv"

# df_a = pd.read_csv(file_a)

# def eda_postchat_hbo(df, name="file"):
#     if "condition" not in df.columns:
#         raise ValueError(f"{name}: no 'condition' column found.")
#     if "time_sec_rel" not in df.columns:
#         raise ValueError(f"{name}: no 'time_sec_rel' column found.")
    
#     d = df[df["condition"] == "post_chat"].copy()
#     hbo_cols = get_hbo_columns(d)
#     if len(hbo_cols) == 0:
#         raise ValueError(f"{name}: no HbO columns found after filtering to post_chat.")
    
#     X = d[hbo_cols].astype(float).values
#     t = d["time_sec_rel"].values
    
#     duration_sec = float(t[-1] - t[0]) if len(t) > 1 else 0.0
#     duration_min = duration_sec / 60.0
#     med_dt, hz = estimate_sampling_rate(t)
#     miss = missingness_rate(X)
    
#     print(f"\n=== EDA (post_chat, HbO only): {name} ===")
#     print("Rows (time points):", len(d))
#     print("HbO channels (D):", len(hbo_cols))
#     print(f"Duration: {duration_sec:.2f} sec (~{duration_min:.2f} min)")
#     print(f"Estimated sampling: dt≈{med_dt:.4f}s  ->  {hz:.2f} Hz")
#     print(f"Missingness rate (NaN fraction): {miss:.6f}")
    
#     return d, hbo_cols, X, t

# eda_postchat_hbo(df_a)


=== EDA (post_chat, HbO only): file ===
Rows (time points): 610
HbO channels (D): 22
Duration: 59.87 sec (~1.00 min)
Estimated sampling: dt≈0.0983s  ->  10.17 Hz
Missingness rate (NaN fraction): 0.000000


(         time_sec  S1_D1 hbo  S1_D1 hbr  S1_D3 hbo  S1_D3 hbr  S2_D1 hbo  \
 4880  1072.005120   0.000006  -0.000020  -0.000021  -0.000004  -0.000051   
 4881  1072.103424   0.000007  -0.000021  -0.000019  -0.000002  -0.000051   
 4882  1072.201728   0.000002  -0.000020  -0.000023  -0.000002  -0.000055   
 4883  1072.300032  -0.000007  -0.000018  -0.000032  -0.000004  -0.000061   
 4884  1072.398336  -0.000015  -0.000017  -0.000042  -0.000008  -0.000071   
 ...           ...        ...        ...        ...        ...        ...   
 5485  1131.479040  -0.000008  -0.000021  -0.000034  -0.000011  -0.000005   
 5486  1131.577344  -0.000008  -0.000021  -0.000032  -0.000012  -0.000004   
 5487  1131.675648  -0.000007  -0.000021  -0.000029  -0.000012  -0.000004   
 5488  1131.773952  -0.000006  -0.000020  -0.000026  -0.000012  -0.000004   
 5489  1131.872256  -0.000005  -0.000020  -0.000024  -0.000011  -0.000003   
 
       S2_D1 hbr  S2_D2 hbo  S2_D2 hbr  S2_D4 hbo  ...  subject_id  dyad_i